In [8]:
import pandas as pd
import ast
import torch
import json
import random
from collections import defaultdict
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler




with open('../../data/usernames_with_titles_updated.json', 'r') as file:
    data = json.load(file)
print(data.keys())

usernames_with_data = data

print(f"Total usernames with data: {len(usernames_with_data)}")

# Print the first 3 usernames and their data
for i, (username, items) in enumerate(usernames_with_data.items()):
    if i >= 3:
        break
    print(f"Username: {username}")
    print("Items:")
    for item, count in items:
        print(f"  - {item}: {count}")
        

dict_keys(['!---slappdash---!', '%D0%90', "'sed", '(.Y.)', '(:Julien:)', '(aeropagitica)', '*Kat*', '*drew', '-Midorihana-', '-Ril-', '-asx-', '-jha-', '.:Ajvol:.', '.V.', '.snoopy.', '068152', '069952497a', '0pen$0urce', '10014derek', '1234r00t', '123Hedgehog456', '125.162.150.88', '131.211.210.14', '131.211.210.157', '156.34.142.110', '172', '17Drew', '194.6.242.2', '1948remembered', '1966batfan', '198', '198.110.249.200', '1ForTheMoney', '1of3', '2', '2004-12-29T22:45Z', '202.58.85.8', '203.125.28.98', '203.213.7.131', '205.157.110.11', '207.6.58.42', '211.30.169.132', '216.153.214.94', '21655', '217.33.207.195', '23prootie', '23skidoo', '24.15.158.90', '24.18.171.99', '24.46.201.42', '24.55.228.56', '24ip', '28bytes', '2D', '2help', '2over0', '32X', '33451', '3centsoap', '3leopard', '43?9enter', '4kinnel', '4twenty42o', '4wajzkd02', '578', '5faizan', '64.12.116.133', '68.190.178.87', '69.121.245.182', '69.57.154.67', '6SJ7', '7', '71.106.28.156', '71.112.134.138', '7107delicious', 

In [10]:
#Full batch dataset
from itertools import islice
from tqdm import tqdm

# Get the first 100 users from usernames_with_data (without converting the whole dictionary to a list)
#first_100_users = dict(islice(usernames_with_data.items(), 100))

# Categories list, it looks like you already have this correctly defined
categories = [
    "Academic disciplines", "Business", "Communication", "Concepts", "Culture", "Economy", "Education", "Energy",
    "Engineering", "Entertainment", "Entities", "Food and drink", "Geography", "Government", "Health", "History",
    "Human behavior", "Humanities", "Information", "Internet", "Knowledge", "Language", "Law", "Life", "Lists", "Mass media",
    "Mathematics", "Military", "Nature", "People", "Philosophy", "Politics", "Religion", "Science", "Society", "Sports",
    "Technology", "Time", "Universe", "Other"
]

categoryString = "\n".join([f"{item}" for item in categories])

# Function to classify users in batches of 10
def classify_users_in_batches(usernames_with_data, batch_size=10):
    all_responses = []  # Store all responses
    
    # Loop over the users in batches of 10
    

    for i in tqdm(range(0, len(usernames_with_data), batch_size)):
        sampled_users = list(usernames_with_data.keys())[i:i + batch_size]
        
        # Step 1: Create a formatted string for the LLM input for each batch
        user_articles = []
        for username in sampled_users:
            articles = usernames_with_data[username]
            formatted_articles = [str(article) for article, _ in articles]
            user_articles.append(f"User: {username}\nArticles: {', '.join(formatted_articles)}")
        llm_input = "\n\n".join(user_articles)
        
        # Step 2: Prepare the classification prompt
        content = (
            f"Below is a list of 40 categories:\n{categoryString}\n\n"
            "You will be given a list of users along with their article names. "
            "Your task is to classify each user into one or more of the given categories based on the context and content of their article names. "
            "The response must be in the following format:\n"
            "<user_name>: category1, category2, category3, category4\n\n"
            "Instructions:\n"
            "1. Use only the provided categories.\n"
            "2. Assign at least one category to each user.\n"
            "3. Do not assign more than 4 categories per user.\n"
            "4. Display only the classification in the specified format. Do not include any additional text or explanation."
        )
        
        # Combine content and question
        prompt = [
            {"role": "system", "content": content},
            {"role": "user", "content": llm_input}
        ]
        
        # LLM initialization (adjust this part based on your actual LLM class and API)
        llm = Ollama(model="qwen2.5:14b")
        response = llm.invoke(prompt)
        
        # Step 3: Process the LLM response
        # Assuming the response is a string with classifications per user
        all_responses.append(response)  # Store the responses for each batch
        
        print(f"Batch {i // batch_size + 1} processed.")

    return all_responses

# Call the function to process the users in batches
all_classifications = classify_users_in_batches(usernames_with_data)

# Output the results (you can store or manipulate them as needed)
for batch_response in all_classifications:
    print(batch_response)

  0%|          | 1/782 [00:16<3:34:11, 16.46s/it]

Batch 1 processed.


  0%|          | 2/782 [00:32<3:29:52, 16.14s/it]

Batch 2 processed.


  0%|          | 3/782 [00:50<3:38:45, 16.85s/it]

...

Batch 780 processed.


100%|█████████▉| 781/782 [3:23:28<00:14, 14.91s/it]

Batch 781 processed.


100%|██████████| 782/782 [3:23:35<00:00, 15.62s/it]

Batch 782 processed.
!---slappdash---!: Entertainment, Music, Sports, Culture
%D0%90: Mass media, Lists, Geography, Education
'sed': Television, Cartoons, Culture, Humor
(.Y.): Music, Lists, Technology, Entertainment
(:Julien:): Sports, Politics, Society, History
(aeropagitica): Television, Biography, Science, Technology
*Kat*: Health, Fiction, Education, Law
*drew: Geography, Mass media, Film, Culture
-Midorihana-: Education, Robotics, Philosophy, Society
-Ril-: Religion, History, Culture, Mythology
-asx-: Politics, History, Law, Media

-jha-: Entities

.:Ajvol:.: History, Government, Economy, Society

.V.: Conspiracy theories, Science, Philosophy, Religion

.snoopy.: Sports, Entertainment, People, Culture

068152: Military, Education, Business, Technology

069952497a: Geography, Economics, Literature, Arts

0pen$0urce: Health, Entertainment, Business, Science

10014derek: Chemistry, Geography, Technology, Food and drink

1234r00t: Natural disasters, Politics, People, History
123Hedge

In [11]:
with open('../../data/classifications_new_user.txt', 'w') as text_file:
    for batch_response in all_classifications:
        text_file.write(f"{batch_response}\n\n")

In [12]:
#process text file
import re

users = list(usernames_with_data.keys())

# Path to your text file
file_path = '../../data/classifications_new_user.txt'

# Initialize an empty dictionary to store usernames and categories
user_categories = {}

# Read the text file
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

    # Loop through each line in the text file
    for line in lines:
        # Remove leading/trailing whitespace
        line = line.strip()
        line = line.replace("User: ", "")
        line = line.replace("User:", "")
        line = line.replace("User", "")
        line = line.strip()

        # Check if the line contains a username from the list
        for username in users:
            if username in line:
                # Extract categories (after the first colon `:`)
                # Use regular expression to split categories, handling cases where `:` might be followed by spaces
                match = re.match(r"([^\:]+)\:(.*)", line)
                if match:
                    found_username = match.group(1).strip()  # Get username (before the colon)
                    categories_str = match.group(2).strip()  # Get categories (after the colon)

                    # Clean up the categories string (remove extra spaces and split by commas)
                    categories = [cat.strip() for cat in categories_str.split(',')]

                    # Store the username and categories in the dictionary
                    user_categories[found_username] = categories

# Now `user_categories` contains the usernames and their associated categories
print(user_categories)

#verification
missing_users = [user for user in users if user not in user_categories]
print(len(missing_users))

# 2. Print the result
if not missing_users:
    print("All users are present in the user_categories dictionary.")
else:
    print(f"Missing users: {', '.join(missing_users)}")

{'!---slappdash---!': ['Entertainment', 'Music', 'Sports', 'Culture'], '%D0%90': ['Mass media', 'Lists', 'Geography', 'Education'], "'sed'": ['Television', 'Cartoons', 'Culture', 'Humor'], '(.Y.)': ['Music', 'Lists', 'Technology', 'Entertainment'], '(': ['Julien:): Sports', 'Politics', 'Society', 'History'], '(aeropagitica)': ['Television', 'Biography', 'Science', 'Technology'], '*Kat*': ['Health', 'Fiction', 'Education', 'Law'], '*drew': ['Geography', 'Mass media', 'Film', 'Culture'], '-Midorihana-': ['Education', 'Robotics', 'Philosophy', 'Society'], '-Ril-': ['Religion', 'History', 'Culture', 'Mythology'], '-asx-': ['Politics', 'History', 'Law', 'Media'], '-jha-': ['Entities'], '.': ['Ajvol:.: History', 'Government', 'Economy', 'Society'], '.V.': ['Conspiracy theories', 'Science', 'Philosophy', 'Religion'], '.snoopy.': ['Sports', 'Entertainment', 'People', 'Culture'], '068152': ['Military', 'Education', 'Business', 'Technology'], '069952497a': ['Geography', 'Economics', 'Literature'

In [13]:
with open("../../data/user_categories_new_new_woohoo.json", "w") as json_file:
    json.dump(user_categories, json_file, indent=4) 